## Базовая обработка таблицы

In [1]:
import pandas as pd
import numpy as np
import statsmodels.stats.api as snss
from scipy import stats
import seaborn as sns

In [2]:
raw_data = pd.read_excel('D:/acces_log_drom.xlsx')
DF = raw_data.copy()
print(DF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100274 entries, 0 to 100273
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   dt            100274 non-null  datetime64[ns]
 1   tme           100274 non-null  datetime64[ns]
 2   host          100274 non-null  object        
 3   uri           100274 non-null  object        
 4   uuid          100274 non-null  object        
 5   http_referer  74413 non-null   object        
dtypes: datetime64[ns](2), object(4)
memory usage: 4.6+ MB
None


In [3]:
DF

,dt,tme,host,uri,uuid,http_referer
0,2023-03-18,2023-03-17 23:59:55,www.drom.ru,/reviews/,0F69F79A9F6D92F0596B101D2C6358BC4,https://www.drom.ru/
1,2023-03-18,2023-03-17 23:59:53,www.drom.ru,/reviews/toyota/,58DDB68605BB0002A50F5A0F8371E0DC1,NaN
2,2023-03-18,2023-03-17 23:59:36,www.drom.ru,/reviews/mazda/,07BCC60A11A489DFA2023C8C1A8AF5F25,NaN
3,2023-03-18,2023-03-17 23:59:34,www.drom.ru,/reviews/byd/,05515201AA597EC7EEABF7B9363914938,NaN
4,2023-03-18,2023-03-17 23:59:16,www.drom.ru,/reviews/mazda/,0901F56AD921645244F1E305473E98000,NaN
...,...,...,...,...,...,...
100269,2023-03-11,2023-03-11 00:00:26,www.drom.ru,/reviews/,DE4666CDBA2501A44395BCA14DFB60C63,https://www.drom.ru/reviews/updates/
100270,2023-03-11,2023-03-11 00:00:25,www.drom.ru,/reviews/,CF22B8FD2BD4B8BB5875DC11B8C1E1BF7,NaN
100271,2023-03-11,2023-03-11 00:00:08,www.drom.ru,/reviews/kia/,FCF6A061C75A9CFA3131AB802E15F8A48,NaN
100272,2023-03-11,2023-03-11 00:00:08,www.drom.ru,/reviews/,8A97E38D55EDEBE2C60390C36956837E7,NaN


In [4]:
DF_test = DF.drop(columns=['dt', 'host'], axis=1) # Создаём новый df без столбцов с датой выгрызки на сервер и именем хоста

In [5]:
#создаем столбец с флагом группы пользователя: 0 - контрольная группа, 1 - тестовая группа
DF_test['group'] = DF_test['uuid'].apply(lambda x: int(x,base=16)) % 2 

In [6]:
DF_test["date"] = DF_test["tme"].dt.to_period("D") # добавление столбца только с датой без времени, для более удобной группировки
DF_test['uuid'] = DF_test['uuid'].apply(int, base=16) # перевод столбца uuid в десятичную систему
DF_test = DF_test.drop_duplicates() # Удаление дублей на основе всей таблицы
DF_test = DF_test.drop(columns=['tme'], axis=1) #удаление столбца с датой в старом формате

In [7]:
# Создаём отдельные dataframe для подсчёта будущих метрик
metric_1 = DF_test.copy()
metric_2 = DF_test.copy()
metric_3 = DF_test.copy()

In [11]:
DF_test # Смотрим на табличку, чтоб не забывать, как она выглядит

,uri,uuid,http_referer,group,date
0,/reviews/,327818129801113720463291375532071291844,https://www.drom.ru/,0,2023-03-17
1,/reviews/toyota/,1889972212027262871408650083813591748033,NaN,1,2023-03-17
2,/reviews/mazda/,164556231915876271500214510578935226149,NaN,1,2023-03-17
3,/reviews/byd/,113094069024035314040327914757957962040,NaN,0,2023-03-17
4,/reviews/mazda/,191571550634552228681038579065592315904,NaN,0,2023-03-17
...,...,...,...,...,...
100269,/reviews/,4727266575192169591165011078636209835107,https://www.drom.ru/reviews/updates/,1,2023-03-11
100270,/reviews/,4405287763877199646625886204303595412471,NaN,1,2023-03-11
100271,/reviews/kia/,5379936206357820436830425500229356718664,NaN,0,2023-03-11
100272,/reviews/,2947553848779231169064165698792552151015,NaN,1,2023-03-11


## Общая посещаемость раздела с отзывами

In [254]:
def uri(link):                         # Функция для обработки ссылки, на которую перешёл пользователь
    if link == '/reviews/':              
        return 1
    else:
        return 0

In [255]:
metric_1['converted'] = metric_1['uri'].apply(uri) # Создаем новый столбец, где 1 - посетил раздел с отзывами, 0 - не посетил

In [256]:
m1_mask1 = (metric_1['group'] == 0)                    # Количество перешедших и их общее количество для контрольной группы
m1_conversions_control = metric_1['converted'][m1_mask1].sum()
m1_total_user_control = metric_1['converted'][m1_mask1].count()
print(m1_conversions_control, m1_total_user_control)

27814 49710


In [257]:
m1_mask2 = (metric_1['group'] == 1)                       # Количество перешедших и их общее количество для тестовой группы
m1_conversions_test = metric_1['converted'][m1_mask2].sum()
m1_total_user_test = metric_1['converted'][m1_mask2].count()
print(m1_conversions_test, m1_total_user_test)

28473 50157


In [258]:
print('Соотношение контрольной и тестовой группы:', round(m1_total_user_control / metric_1['converted'].count() * 100, 2), "%",
      round(m1_total_user_test / metric_1['converted'].count() * 100, 2), "%")

Соотношение контрольной и тестовой группы: 49.78 % 50.22 %


In [259]:
count = np.array([m1_conversions_control, m1_conversions_test])                          # Z_статистика
nobs = np.array([m1_total_user_control, m1_total_user_test])
z_score, p_value = sns.proportions_ztest(count, nobs, value=0, alternative='two-sided')

In [260]:
print('p-value:', p_value)
p1 = p_value

p-value: 0.009394972687726509


In [261]:
print(p1, 'Первая метрика')
print(p1 > 0.05)

0.009394972687726509 Первая метрика
False


## Ретеншн 2-го дня

In [263]:
metric_2 = metric_2.drop_duplicates(subset=['year_day', 'uuid']) # Фильтрация уникальных значений uuid для каждого дня

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55073 entries, 0 to 100272
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   tme           55073 non-null  datetime64[ns]
 1   uri           55073 non-null  object        
 2   uuid          55073 non-null  object        
 3   http_referer  35003 non-null  object        
 4   group         55073 non-null  object        
 5   year_day      55073 non-null  period[D]     
dtypes: datetime64[ns](1), object(4), period[D](1)
memory usage: 2.9+ MB
None


In [264]:
metric_2_first_date = metric_2['uuid'][metric_2['year_day'] == '2023-03-11']  # уникальные пользователи 2023-03-11

In [265]:
metric_2_second_date = metric_2['uuid'][metric_2['year_day'] == '2023-03-13'] # уникальные пользователи 2023-03-13

In [266]:
def converted(uuid):                      # Функция для нахождения одинаковых uuid 2023-03-11 и 2023-03-13
    if uuid in list(metric_2_second_date):
        return 1
    else:
        return 0

In [267]:
metric_2['converted'] = metric_2_first_date.apply(converted) # новый столбец, где 1 - юзер вернулся через 2 дня, 0 - не вернулся

In [269]:
m2_mask1 = (metric_2['group'] == 0)                     # Количество перешедших и их общее количество для контрольной группы
m2_conversions_control = metric_2['converted'][m2_mask1].sum()
m2_total_user_control = metric_2['converted'][m2_mask1].count()
print(m2_conversions_control, m2_total_user_control)

387.0 4595


In [270]:
m2_mask2 = (metric_2['group'] == 1)                      # Количество перешедших и их общее количество для тестовой группы
m2_conversions_test = metric_2['converted'][m2_mask2].sum()
m2_total_user_test = metric_2['converted'][m2_mask2].count()
print(m2_conversions_test, m2_total_user_test)

374.0 4495


In [271]:
print('Соотношение контрольной и тестовой группы:', round(m2_total_user_control / metric_2['converted'].count() * 100, 2), "%",
      round(m2_total_user_test / metric_2['converted'].count() * 100, 2), "%")

Соотношение контрольной и тестовой группы: 50.55 % 49.45 %


In [272]:
count = np.array([m2_conversions_control, m2_conversions_test])                         # Z_статистика
nobs = np.array([m2_total_user_control, m2_total_user_test])
z_score, p_value = sns.proportions_ztest(count, nobs, value=0, alternative='two-sided')

In [273]:
print('p-value:', p_value)
p2 = p_value

p-value: 0.8608609808530691


In [274]:
print(p2, 'Вторая метрика')
print(p2 > 0.05)

0.8608609808530691 Вторая метрика
True


## Конверсия с главной страницы отзывов

In [8]:
HTTP_REFERER = 'https://www.drom.ru/reviews/'  # создаём переменную со значением реферальной ссылки
                                                                # и словарь со всеми марками автомобилей
BRAND = ['ac', 'aito', 'acura', 'alfa_romeo', 'alpina', 'aro', 'asia', 'aston_martin', 'audi', 'baic', 'baw', 'bmw',
         'byd', 'bentley', 'brilliance', 'bugatti', 'buick', 'cadillac', 'changan', 'chery', 'chevrolet', 'chrysler',
         'citroen', 'cupra', 'dw_hower', 'dacia', 'dadi', 'daewoo', 'daihatsu', 'daimler', 'datsun', 'delorean',
         'derways', 'dodge', 'dongfeng', 'cheryexeed', 'eagle', 'evolute', 'faw', 'ferrari', 'fiat', 'fisker', 'ford',
         'foton', 'freightliner', 'gac', 'gmc', 'geely', 'genesis', 'geo', 'great_wall', 'hafei', 'haima', 'haval',
         'hawtai', 'hino', 'honda', 'hongqi', 'howo', 'hummer', 'hyundai', 'infiniti', 'iran_khodro', 'isuzu', 'jac',
         'jaguar', 'jeep', 'jetour', 'kaiyi', 'kia', 'koenigsegg', 'lamborghini', 'lancia', 'land_rover', 'lexus',
         'li', 'lifan', 'lincoln', 'lotus', 'luxgen', 'mini', 'marussia', 'maserati', 'maxus', 'maybach', 'mazda',
         'mclaren', 'mercedes-benz', 'mercury', 'mitsubishi', 'mitsuoka', 'nissan', 'omoda', 'oldsmobile', 'opel',
         'pagani', 'peugeot', 'plymouth', 'pontiac', 'porsche', 'proton', 'ram', 'ravon', 'renault', 'renault_samsung',
         'rivian', 'rolls-royce', 'rover', 'seat', 'saab', 'saturn', 'scion', 'skoda', 'skywell', 'smart', 'ssang_yong',
         'subaru', 'suzuki', 'tvr', 'tank', 'tesla', 'tianye', 'toyota', 'volkswagen', 'volvo', 'vortex', 'weltmeister',
         'wiesmann', 'xin_kai', 'zx', 'zotye', 'bogdan', 'gaz', 'zaz', 'zil', 'izh', 'lada', 'luaz', 'moskvitch',
         'tagaz', 'uaz']

In [9]:
metric_3 = metric_3.loc[DF['http_referer'] == HTTP_REFERER] # Фильтрация таблицы по http_referer

In [10]:
def uri(link):                 # Функция для обработки ссылки
    link = str(link)
    list1 = link.split("/")
    list1 = list(filter(None, list1))
    if 'reviews' in list1 and list1[-1] in BRAND and list1[-2] not in BRAND:
        return 1
    else:
        return 0

In [11]:
metric_3['converted'] = metric_3['uri'].apply(uri) # Создаем новый столбец, где 1 - есть конверсия, 0 - нет

In [12]:
m3_mask1 = (metric_3['group'] == 0)              # Количество перешедших и их общее количество для контрольной группы
m3_conversion_0 = metric_3['converted'][m3_mask1].to_numpy() # Создаём нампаевский массив для с конверсией для контрольной группы
m3_conversions_control = metric_3['converted'][m3_mask1].sum()
m3_total_user_control = metric_3['converted'][m3_mask1].count()
print(m3_conversions_control, m3_total_user_control)

9512 11706


In [13]:
m3_mask2 = (metric_3['group'] == 1)                  # Количество перешедших и их общее количество для тестовой группы
m3_conversion_1 = metric_3['converted'][m3_mask2].to_numpy() # Создаём нампаевский массив для с конверсией для тестовой группы
m3_conversions_test = metric_3['converted'][m3_mask2].sum()
m3_total_user_test = metric_3['converted'][m3_mask2].count()
print(m3_conversions_test, m3_total_user_test)

9279 11607


In [14]:
print('Соотношение контрольной и тестовой группы:', round(m3_total_user_control / metric_3['converted'].count() * 100, 2), "%",
      round(m3_total_user_test / metric_3['converted'].count() * 100, 2), "%")

Соотношение контрольной и тестовой группы: 50.21 % 49.79 %


In [15]:
def bootstrap(m3_conversion_0, m3_conversion_1, n_bootstrap=2000): #Создаём функцию bootstrap

    poisson_bootstraps1 = stats.poisson(1).rvs(
        (2000, len(m3_conversion_0))).astype(np.int64)

    poisson_bootstraps2 = stats.poisson(1).rvs(
            (2000, len(m3_conversion_1))).astype(np.int64)
    
    conversion_control = (poisson_bootstraps1*m3_conversion_0).sum(axis=1) / len(m3_conversion_0)
    
    conversion_test = (poisson_bootstraps2*m3_conversion_1).sum(axis=1) / len(m3_conversion_1)

    return conversion_control, conversion_test

In [16]:
cnvrs0, cnvrs1 = bootstrap(m3_conversion_0, m3_conversion_1)

In [ ]:
cnvrs0.to_exel()

In [97]:
z_score, p_value = snss.proportions_ztest(cnvrs0, cnvrs1, value=0, alternative='two-sided')

NotImplementedError: more than two samples are not implemented yet

In [90]:
stats.ttest_ind(cnvrs0,      # Применяем Т-тест
                cnvrs1,
                equal_var=False)

Ttest_indResult(statistic=49.74818318142018, pvalue=0.0)

In [84]:
cnvrs0.mean(), cnvrs1.mean() # 0.812517127968563, 0.7991309123804601

(0.8125046984452418, 0.7992004393900234)

In [283]:
print('p-value:', p_value)
p3 = p_value

p-value: 0.011160301156558543


In [284]:
print(p3, 'Вторая метрика')
print(p3 > 0.05)

0.011160301156558543 Вторая метрика
False


## Поправка Бонферрони

In [287]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [285]:
pval = [p1, p2, p3]
print(pval)

[0.009394972687726509, 0.8608609808530691, 0.011160301156558543]


In [288]:
rejected, p_corrected, _, _ = multipletests(pval, alpha=0.05, method='bonferroni')
print(p_corrected)
print(p_corrected > 0.05)

[0.02818492 1.         0.0334809 ]
[False  True False]
